In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0, 1, 2, 3'

import json
import pickle
import sparse
import random
import librosa
import mir_eval
import fluidsynth
import pretty_midi
import numpy as np
from time import time

import IPython.display
import matplotlib.pyplot as plt

import torch
import torch.optim as optim
import torch.nn.functional as F

from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import RandomSampler
from torch.utils.data.distributed import DistributedSampler

from utils.drum_utils import *
from utils.common_utils import *
from utils.train import *
from utils.test import *
from utils.layers import *
from utils.loss import *

%load_ext autoreload
%autoreload 2

In [ ]:
# initialize model with GPU
use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')

### Load data

In [ ]:
path = '../data/Lakh_9drum_4bar_4of4_16quant_round_big.pkl'
with open(path, 'rb') as f:
    data = pickle.load(f)

print('The number of data : %d' % len(data))

In [ ]:
# play example
fs = 7
pm = drum_play(data[814].todense(), fs)
IPython.display.Audio(pm.fluidsynth(fs=16000), rate=16000)

In [ ]:
# shuffle and split
num_data = len(data)
random.shuffle(data)

num_train = int(num_data * 0.7)
num_val = int(num_data * 0.1)

train_data = data[:num_train]
val_data = data[num_train:num_train+num_val]
test_data = data[num_train+num_val:]

print('The number of train: %d' % len(train_data))
print('The number of validation: %d' % len(val_data))
print('The number of test: %d' % len(test_data))

In [ ]:
# custom dataloader
class DatasetSampler(Dataset):
    def __init__(self, x):
        self.x = x

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx].todense().astype('float32')

In [ ]:
# dataloader
params = {'batch_size': 1024*4, 
          'shuffle': True,
          'pin_memory': True,
          'num_workers': 4}

train_set = DataLoader(DatasetSampler(train_data), **params)
val_set = DataLoader(DatasetSampler(val_data), **params)
test_set = DataLoader(DatasetSampler(test_data), **params)

In [ ]:
# init model
enc_input_size = 512
enc_hidden_size = 1024
enc_latent_dim = 512

encoder = Encoder(enc_input_size, enc_hidden_size, enc_latent_dim)
encoder = nn.DataParallel(encoder).to(device)

dec_input_size = 512
dec_hidden_size = 1024
dec_output_size = 512

decoder = Decoder(dec_input_size, dec_hidden_size, dec_output_size)
decoder = decoder.to(device)

model = [encoder, decoder]

In [ ]:
# init optimizer
enc_optimizer = optim.Adam(encoder.parameters(), lr=1e-3)
dec_optimizer = optim.Adam(decoder.parameters(), lr=1e-3)
optimizer = [enc_optimizer, dec_optimizer]

### Train

In [ ]:
history = flat_train(device, vae_loss, train_set, val_set, model, optimizer, epochs=1)

In [ ]:
# # save model
# path = '../model/Lakh_9drum_4bar_4of4_16quant_floor'
# torch.save(encoder.state_dict(), path+'_encoder.pt')
# torch.save(decoder.state_dict(), path+'_decoder.pt')

# # load model
# model = TheModelClass(*args, **kwargs)
# model.load_state_dict(torch.load(PATH))
# model.eval()

### Test

In [ ]:
history, y_true, y_pred = flat_test(device, vae_loss, test_set, model, temp=1, options='full_sampling')

In [ ]:
# input
fs = 7; idx = 255
pm = drum_play(y_true[idx], fs)
IPython.display.Audio(pm.fluidsynth(fs=16000), rate=16000)

In [ ]:
# reconstruct sampled from categorical distribution
pm = drum_play(prob_soft_label(y_pred[idx]), fs)
IPython.display.Audio(pm.fluidsynth(fs=16000), rate=16000)
# pm.write('output.mid')

### Generate custom inputs and predict

In [ ]:
def predict(z, decoder, seq_len=64, temp=1):
    batch_size = z.shape[0]
    
    hidden_size = decoder.hidden_size
    output_size = decoder.output_size
    num_hidden = decoder.num_hidden
    
    h = z.repeat(num_hidden, 1, int(hidden_size/z.shape[1]))
    c = z.repeat(num_hidden, 1, int(hidden_size/z.shape[1]))
    
    inputs = torch.zeros((batch_size, 1, output_size), device=device)
    inputs = torch.cat((inputs, z.unsqueeze(1)), 2)
    outputs = torch.zeros((batch_size, seq_len, output_size), device=device) # argmax
    
    # full sampling
    for j in range(seq_len):
        label, prob, h, c = decoder(inputs, h, c, temp=temp)
        outputs[:, j, :] = prob.squeeze()

        label = F.one_hot(label, num_classes=512)
        inputs = torch.cat((label, z.unsqueeze(1)), 2)
        
    return outputs

In [ ]:
# custom input
sequence = [[0, 3], [3], [3], [0, 3], [3], [3], [1, 3], [3]]

In [ ]:
# generate new sample
dim = 512
hot_encoding = np.eye(dim)
hot_encoded = np.zeros((2*len(sequence), dim), dtype='float32')

for i in range(0, 2*len(sequence), 2):
    hit_idx = int(i/2)
    if sequence[hit_idx][0] == -1:
        hot_encoded[i, 0] = 1
        continue
        
    temp = np.zeros(9)
    temp[sequence[hit_idx]] = 1
    decimal = bin_to_dec(temp)
    
    hot_encoded[i, :] = hot_encoding[decimal]
    hot_encoded[i+1, 0] = 1 # rest
    
hot_encoded = np.tile(hot_encoded, (4, 1))
print('input shape :', hot_encoded.shape)

In [ ]:
# play input
fs = 7
pm = drum_play(hot_encoded, fs)
IPython.display.Audio(pm.fluidsynth(fs=16000), rate=16000)

In [ ]:
# MusicVAE inference
fs = 7
temp = 2

test = torch.from_numpy(hot_encoded).to(device).unsqueeze(0)

z, mu, std = encoder(test)
pred = np.squeeze(predict(z, decoder, temp=temp).data.cpu().numpy())

pm = drum_play(prob_soft_label(pred), fs=fs)
IPython.display.Audio(pm.fluidsynth(fs=16000), rate=16000)